In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import slim
CIFAR_DIR = "../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))
import matplotlib.pyplot as plt

['.ipynb_checkpoints', '0.py', '2.csv', '3.csv', '3chTest', '3chTest1', '3chTrain', '3chTrain1', '3chTrain2', '4.csv', '5.csv', 'a.jpg', 'batches.meta', 'cDCGAN_VS_CNN-Copy1.ipynb', 'cDCGAN_VS_CNN.ipynb', 'checkpoint', 'data', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'imageChangeChannel.ipynb', 'readme.html', 'resize.py', 'splite_data.py', 'SSGAN_VS_CNN_calculate_confusion_matrix.ipynb', 'test', 'test.png', 'test1.png', 'test2', 'test2.png', 'test3', 'testNew', 'test_batch', 'train', 'train2', 'train3', 'trainNew', 'Untitled-Copy1.ipynb', 'Untitled1.ipynb']


In [2]:
import os 
import numpy as np
from PIL import Image
def load_Traindata():
    train = []
    file_names = os.listdir('./3chTrain1/')
    for i in range(len(file_names)):
        im = Image.open('./3chTrain1/'+file_names[i])
        im2 = np.array(im)
        train.append(im2)
    train2 = np.array(train)
    train_lable = np.zeros((len(train2),1),dtype='int8')
    for i in range(6):
        for j in range(i*1711, (i+1)*1711):
            train_lable[j,0] = i
   
    train_lable = train_lable.reshape((train_lable.shape[0],))
    return (train2, train_lable)
def load_Testdata():
    test = []
    file_names = os.listdir('./3chTest1/')
    for i in range(len(file_names)):
        im = Image.open('./3chTest1/'+file_names[i])
        im2 = np.array(im)
        test.append(im2)
    test2 = np.array(test)
    test_lable = np.zeros((len(test2),1),dtype='int8')
    for i in range(6):
        for j in range(i*435, (i+1)*435):
            test_lable[j,0] = i
    test_lable = test_lable.reshape((test_lable.shape[0],))
    return (test2, test_lable)

In [3]:
def load(checkpoint_dir, saver, sess):
    import re
    print(" [*] Reading checkpoints...")
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
        counter = int(next(re.finditer("(\d+)(?!.*\d)", ckpt_name)).group(0))
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0


def save(checkpoint_dir, step, saver, sess):
    model_name = "cDCGAN.model"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver.save(sess, os.path.join(checkpoint_dir, model_name), global_step=step)

In [37]:
data_len = 0
path = 'D:/cifar-10-batches-py/3chTest1'

for lists in os.listdir(path):
    sub_path = os.path.join(path, lists)
    if os.path.isfile(sub_path):
        data_len = data_len+1
class SteelData:
    def __init__(self,  need_shuffle, isTrain):
        all_data = []
        all_labels = []
        if isTrain:
            data, labels = load_Traindata()
        else:
            data, labels = load_Testdata()
        all_data.append(data)
        all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def get_test_batch(self):
        """return batch_size examples as a batch."""
        for i in range(6):
            # 训练集打乱顺序随机输入
            data_len_per_class = data_len//6
            order = np.random.permutation(data_len//6)           
            if i == 0:
                for k in range(60):
                    if k == 0:
                        batch_data = self._data[(i*data_len_per_class+order[0])]
                        batch_data = np.reshape(batch_data, (1, 64, 64, 3))
                        batch_labels = self._labels[(i*data_len_per_class+order[0])]
                    else:
                        batch_data = np.concatenate((batch_data,np.reshape(self._data[(i*data_len_per_class+order[k])], (1, 64, 64, 3))), axis=0)
                        batch_labels = np.append(batch_labels,self._labels[(i*data_len_per_class+order[0])])
            else:
                for k in range(60):
                        batch_data = np.concatenate((batch_data,np.reshape(self._data[(i*data_len_per_class+order[k])], (1, 64, 64, 3))), axis=0)
                        batch_labels = np.append(batch_labels,self._labels[(i*data_len_per_class+order[k])])
        return batch_data, batch_labels

train_data = SteelData(True, True)
test_data = SteelData(False, False)

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 64, 64, 3])
#x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# [None], eg: [0,5,6,3]
#x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
#x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

with tf.variable_scope('cDCGAN', reuse=tf.AUTO_REUSE):
    w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    cnum = 64

    conv1 = tf.layers.conv2d(x, cnum, 5, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv1')
    bn1 = tf.layers.batch_normalization(conv1)
    conv2 = tf.layers.conv2d(bn1, 2 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv2')
    bn2 = tf.layers.batch_normalization(conv2)
    conv3 = tf.layers.conv2d(bn2, 4 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv3')
    bn3 = tf.layers.batch_normalization(conv3)
    conv4 = tf.layers.conv2d(bn3, 8 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv4')
    # output real or false
    fla = tf.layers.flatten(conv4, name='flatten')
    fc1 = tf.layers.dense(fla, 1024, name='fc1')


                # output cls
                # 共享fc1层
    y_ = tf.layers.dense(fc1, 6, name='cls')
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
init = tf.global_variables_initializer()
batch_size = 36
train_steps = 1000
test_steps = 10

# train 10k: 71.35%
# with tf.Session() as sess:
#     sess.run(init)
#     all_steps = []
#     all_train_acc = []
#     all_test_steps = []
#     all_test_acc = []
#     all_test_acc_val = []
#     all_loss = []
#     saver = tf.train.Saver()
#     #show_all_variables()
#     # log dir
#     #summary_op = tf.summary.merge_all()

#     #writer = tf.summary.FileWriter("./logs/" + '_'.join(
#         #[config.DATASET_TRAIN, config.MODEL_NAME]), sess.graph)

#     log_prefix = './checkpoint' + '/' + 'train_cDCGAN'
#     counter = 1
#     #start_time = time.time()
#     could_load, checkpoint_counter = load(log_prefix, saver, sess)
#     if could_load:
#         counter = checkpoint_counter
#         print(" [*] Load SUCCESS")
#     else:
#         print(" [!] Load failed...")
#     for i in range(train_steps):
        
        
        
#         batch_data, batch_labels = train_data.next_batch(batch_size)
#         loss_val, acc_val, _ = sess.run(
#             [loss, accuracy, train_op],
#             feed_dict={
#                 x: batch_data,
#                 y: batch_labels})
#         if (i+1) % 50 == 0:
#             save(log_prefix, counter, saver, sess)
#             print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
#                   % (i+1, loss_val, acc_val))
#             all_steps.append(i+1)
#             all_train_acc.append(acc_val)
#             all_loss.append(loss_val)
#             #if (i+1) % 1000 == 0:
#             test_data = SteelData(False,False)


#             test_batch_data, test_batch_labels \
#                         = test_data.next_batch(1)
#             test_acc_val = sess.run(
#                         [accuracy],
#                         feed_dict = {
#                             x: test_batch_data, 
#                             y: test_batch_labels
#                         })
#             all_test_acc_val.append(test_acc_val)

#             all_test_steps.append(i)
#             #test_acc = np.mean(all_test_acc_val)
#             test_acc =  np.mean(test_acc_val)
#         #all_test_acc.append(test_acc)
#             print('[Test ] Step: %d, acc: %4.5f'
#                   % (i+1, test_acc))
#             counter += 1


with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver()
    log_prefix = './checkpoint' + '/' + 'train_cDCGAN'
    counter = 1
        #start_time = time.time()
    could_load, checkpoint_counter = load(log_prefix, saver, sess)
    if could_load:
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        print(" [!] Load failed...")
      
    for i in range(train_steps):
#         batch_data, batch_labels = train_data.next_batch(batch_size)
#         loss_val, acc_val, _ = sess.run(
#             [loss, accuracy, train_op],
#             feed_dict={
#                 x: batch_data,
#                 y: batch_labels})
#         if (i+1) % 100 == 0:
#             print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
#                   % (i+1, loss_val, acc_val))
        
        test_data = SteelData(False, False)
        all_test_acc_val = []
        for j in range(test_steps):
            test_batch_data, test_batch_labels \
                    = test_data.get_test_batch()
            test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
            all_test_acc_val.append(test_acc_val)
        test_acc = np.mean(all_test_acc_val)
        confusion_matrix = tf.confusion_matrix(predictions=predict,
                                               labels=test_batch_labels,
                                               num_classes=6, dtype=tf.int32,
                                               name=None, weights=None)
        confusion_matrix1 = sess.run(confusion_matrix, feed_dict={
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
        print(confusion_matrix1)
        print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))
        
    all_test_acc_val = []
     
    #print(all_steps)      
    #print(all_train_acc)
#     fig,ax = plt.subplots()
 
#     #plt.xlabel('migration speed (MB/s)')
#     #plt.ylabel('migration time (s); request delay (ms)')

#     """set interval for y label"""
#     #yticks = range(0,1,11)
#     #ax.set_yticks(yticks)


#     """set min and max value for axes"""
#     #ax.set_ylim([0,1])
#     ax.set_xlim([0,10000])


#     #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
#     line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
#     line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
#     line3, =plt.plot(all_steps,all_loss,'--',label="test accrucy")
#     #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

#     """open the grid"""
#     plt.grid(True)

#     #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
#     plt.legend([line1, line2,line3], ['train accrucy', 'test accrucy','loss'], loc = 'best') 
#     #plt.savefig('./test.png')
#     plt.savefig('./test1.png')
#     plt.show()




 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ./checkpoint/train_cDCGAN\cDCGAN.model-14
 [*] Success to read cDCGAN.model-14
 [*] Load SUCCESS
[[56  0  3  1  0  0]
 [ 0 35  6  8  1 10]
 [ 0  0 60  0  0  0]
 [ 1 17  0 42  0  0]
 [ 0  2  0  0 58  0]
 [ 0  1  0  0  0 59]]
[Test ] Step: 1, acc: 0.86500
[[56  0  2  1  1  0]
 [ 0 37  6 12  0  5]
 [ 0  0 59  0  0  1]
 [ 0 16  0 44  0  0]
 [ 0  2  0  0 58  0]
 [ 0  0  0  0  0 60]]
[Test ] Step: 2, acc: 0.87083
[[55  0  2  0  3  0]
 [ 0 29  6 13  3  9]
 [ 0  0 59  0  0  1]
 [ 2 21  0 37  0  0]
 [ 0  1  0  0 59  0]
 [ 0  1  1  0  0 58]]
[Test ] Step: 3, acc: 0.86639


In [50]:
print(all_test_acc_val)

[[0.8888888888888888], [0.7222222222222222], [0.9444444444444444], [0.8333333333333334], [0.8611111111111112], [0.9166666666666666], [0.8055555555555556], [0.8888888888888888], [0.8333333333333334], [0.8888888888888888]]


In [ ]:
fig,ax = plt.subplots()
 
#plt.xlabel('migration speed (MB/s)')
    #plt.ylabel('migration time (s); request delay (ms)')

    #"""set interval for y label"""
    #yticks = range(0,1,11)
    #ax.set_yticks(yticks)


    #"""set min and max value for axes"""
ax.set_ylim([0,1.1])
ax.set_xlim([0,10000])


    #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
    #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

 #"""open the grid"""
plt.grid(True)

    #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
plt.legend([line1, line2], ['train accrucy', 'test accrucy'], loc = 'best') 
plt.savefig('./test.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
 
#plt.xlabel('migration speed (MB/s)')
    #plt.ylabel('migration time (s); request delay (ms)')

    #"""set interval for y label"""
    #yticks = range(0,1,11)
    #ax.set_yticks(yticks)


    #"""set min and max value for axes"""
#ax.set_ylim([0,1.1])
ax.set_xlim([0,10000])


    #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
#line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
#line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
    #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

 #"""open the grid"""
plt.grid(True)
plt.plot(all_steps,all_loss,'--',label="test accrucy")
    #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
#plt.legend([line1, line2], ['train accrucy', 'test accrucy'], loc = 'best') 
plt.savefig('./test2.png')
plt.show()

In [8]:
import csv

In [9]:
row1 = all_train_acc
print(row1)

[0.3333333333333333, 0.5, 0.4722222222222222, 0.4722222222222222, 0.5833333333333334, 0.8333333333333334, 0.7222222222222222, 0.75, 0.7222222222222222, 0.7777777777777778, 0.8888888888888888, 0.9444444444444444, 0.9722222222222222, 0.8888888888888888, 0.8888888888888888, 0.9444444444444444, 0.9166666666666666, 0.8333333333333334, 0.8888888888888888, 1.0, 0.8333333333333334, 1.0, 0.9444444444444444, 0.8333333333333334, 0.9166666666666666, 0.9444444444444444, 0.8611111111111112, 0.9166666666666666, 1.0, 1.0, 0.9722222222222222, 0.8888888888888888, 0.8888888888888888, 0.8611111111111112, 0.8611111111111112, 1.0, 0.9722222222222222, 0.9166666666666666, 0.8611111111111112, 0.9722222222222222, 0.9166666666666666, 0.8888888888888888, 0.9722222222222222, 1.0, 0.9166666666666666, 0.9722222222222222, 1.0, 0.9722222222222222, 0.9722222222222222, 0.9444444444444444, 0.8611111111111112, 0.9444444444444444, 1.0, 0.9722222222222222, 1.0, 0.8888888888888888, 0.9444444444444444, 0.9444444444444444, 0.9

In [10]:
row2 = all_steps

In [11]:
data = list(map(lambda x:[x],row1))
with open('4.csv','w') as f:
    f_csv = csv.writer(f)
    for i in data:
            f_csv.writerow(i)
    #f_csv.writerows(row2)
    #f_csv.writerows(row1)

In [ ]:
np.mean(all_test_acc_val[-6:])

In [12]:
data = list(map(lambda x:[x],row2))
with open('5.csv','w') as f:
    f_csv = csv.writer(f)
    for i in data:
            f_csv.writerow(i)

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 64, 64, 3])
#x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])


with tf.variable_scope('cDCGAN', reuse=tf.AUTO_REUSE):
    w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    cnum = 64

    conv1 = tf.layers.conv2d(x, cnum, 5, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv1')
    bn1 = tf.layers.batch_normalization(conv1)
    conv2 = tf.layers.conv2d(bn1, 2 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv2')
    bn2 = tf.layers.batch_normalization(conv2)
    conv3 = tf.layers.conv2d(bn2, 4 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv3')
    bn3 = tf.layers.batch_normalization(conv3)
    conv4 = tf.layers.conv2d(bn3, 8 * cnum, 3, 2, padding='same', kernel_initializer=w_init,
                                         bias_initializer=b_init, activation=tf.nn.leaky_relu, name='conv4')
    # output real or false
    fla = tf.layers.flatten(conv4, name='flatten')
    fc1 = tf.layers.dense(fla, 1024, name='fc1')


                # output cls
                # 共享fc1层
    y_ = tf.layers.dense(fc1, 6, name='cls')
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
init = tf.global_variables_initializer()
batch_size = 5
train_steps = 10000
test_steps = 100

train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    all_steps = []
    all_train_acc = []
    all_test_steps = []
    all_test_acc = []
    all_test_acc_val = []
    all_loss = []
    saver = tf.train.Saver()
    #show_all_variables()
    # log dir
    #summary_op = tf.summary.merge_all()

    #writer = tf.summary.FileWriter("./logs/" + '_'.join(
        #[config.DATASET_TRAIN, config.MODEL_NAME]), sess.graph)

    log_prefix = './checkpoint' + '/' + 'train_cDCGAN'
    counter = 1
    #start_time = time.time()
    could_load, checkpoint_counter = load(log_prefix, saver, sess)
    if could_load:
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        print(" [!] Load failed...")
    for i in range(train_steps):
        i = 0
        y_true = []
        y_pred = []
        order = np.random.permutation(data_len)
        for idx in range(5000):
            batch_data_files = []
            batch_label = []
            batch_data_files.append(data_list[order[idx]])
            batch_label.append(label_list[order[idx]])
            #batch_data_files.append(data_list[idx])
            #batch_label.append(label_list[idx])
            batch_data = [
                get_image(batch_file, grayscale=True) for batch_file in batch_data_files]
            batch_images = np.array(batch_data).astype(np.float32)
            batch_images = np.expand_dims(batch_images, axis=3)
            batch_label = np.array(batch_label).astype(np.uint8)

            feed_dict = {images: batch_images,
                         labels: batch_label}
            cl = sess.run(tf.argmax(cls, axis=1), feed_dict=feed_dict)
            y_true.append(batch_label[0])
            y_pred.append(cl[0])
            if cl[0] == batch_label[0]:
                i += 1
            print("Epoch: [%4d], real: %d, d: %d" % (idx, batch_label[0], cl[0]))

        print(i/5000)
        
        

